In [1]:
from pathlib import Path

import numpy as np
from numba import njit

from matplotlib import pyplot as plt

In [2]:
def one_hot(n_classes: int, idx: int) -> np.ndarray:
    embedding = np.zeros(n_classes)
    embedding[idx] = 1.0
    return embedding


def word2seq(word: str, alphabet: list) -> np.ndarray:
    return np.array([one_hot(len(alphabet), alphabet.index(c)) for c in word])


def seq2word(seq: np.ndarray, alphabet: list) -> str:
    return ''.join([alphabet[np.argmax(V)] for V in seq])

In [3]:
alphabet = list('abcdefghijklmnopqrstuvwxyz .,')

In [4]:
@njit(fastmath=True)
def entropy(p: np.ndarray) -> float:
    return -np.sum(p*np.log(p))


@njit(fastmath=True)
def cross_entropy(p: np.ndarray, q: np.ndarray) -> float:
    return -np.sum(p*np.log(q))

In [5]:
@njit(fastmath=True)
def sigmoid(x: np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-x))


@njit(fastmath=True)
def dsigmoid(x: np.ndarray) -> np.ndarray:
    y = 1 / (1 + np.exp(-x))
    return y * (1 - y)


@njit(fastmath=True)
def softmax(x: np.ndarray) -> np.ndarray:
    y = np.exp(x)
    return y / np.sum(y)

### Define model

In [10]:
class RNNClassifier:
    def __init__(self, n_in: int, n_h: int, n_out: int) -> None:
        self.U: np.ndarray = np.random.uniform(-1, 1, (n_h, n_in))
        self.V: np.ndarray = np.random.uniform(-1, 1, (n_h, n_h))
        self.d: np.ndarray = np.zeros(n_h)

        self.W: np.ndarray = np.random.uniform(-1, 1, (n_out, n_h))
        self.b: np.ndarray = np.zeros(n_out)
    
    @property
    def parameters(self) -> tuple[np.ndarray]:
        return (self.U, self.V, self.d, self.W, self.b)

    def forward(self, sequence: np.ndarray) -> np.ndarray:
        h: np.ndarray = np.zeros(len(self.V))
        for x in sequence:
            h = np.tanh(self.U @ x + self.V @ h + self.d)
        return softmax(self.W @ h + self.b)

    def loss(self, sequences: np.ndarray, y: np.ndarray) -> float:
        y_hat = np.array([self.forward(x) for x in sequences])
        return np.mean(np.array([cross_entropy(p, q) for (p, q) in zip(y, y_hat)]))

In [7]:
@njit(fastmath=True)
def grads(parameters: tuple[np.ndarray], x_batch: np.ndarray, y_batch: np.ndarray) -> tuple[np.ndarray]:
    U, V, d, W, b = parameters

    dU, dV, dW = np.zeros(U.shape), np.zeros(V.shape), np.zeros(W.shape)
    dd, db = np.zeros(d.shape), np.zeros(b.shape)

    for i in range(len(x_batch)):
        pass

    return (dU, dV, dd, dW, db)    


def train(model: RNNClassifier, x_train: np.ndarray, y_train: np.ndarray, lr, batch_size, max_epoch) -> None:
    n = x_train.shape[0]
    n_batches = n // batch_size

    for epoch in range(max_epoch):
        idxs = np.random.permutation(n)

        for i in range(n_batches):
            ibegin = i * batch_size
            iend = min((i + 1) * batch_size, n - 1)
            batch_idxs = idxs[ibegin:iend]

            dU, dV, dd, dW, db = grads(
                model.parameters,
                x_train[batch_idxs],
                y_train[batch_idxs]
            )

            model.U -= lr * dU
            model.V -= lr * dV
            model.d -= lr * dd

            model.W -= lr * dW
            model.b -= lr * db

### Create and train model

In [14]:
n_in = len(alphabet)
n_h = 64
n_out = 2

learning_rate = 1e-1
batch_size = 8
max_epoch = 100

model = RNNClassifier(n_in, n_h, n_out)

In [15]:
text = 'the quick brown fox, jumps over the lazy dog'
x = word2seq(text, alphabet)
y = model.forward(x)
print(np.round(y, 3))

[0.004 0.996]
